# Análise da cobertura do projeto Gás para Todos

## Apresentação

Eu me chamo João Ricardo Oliveira. Sou professor de Matemática e Física, e tenho quase 30 anos de experiência ensinando essas duas fascinantes disciplinas. Uma carreira da qual me orgulho, e na qual certamente permaneceria, se não fosse um certo convite.






## Como tudo começou

No início de 2022, um amigo me convidou para trabalhar em um projeto de uma ONG do 3o setor. Achei a proposta interessante. Afinal, era um projeto socialmente relevante, além de ser algo totalmente diferente do que sempre fiz por toda minha carreira. Por isso, apesar de relutante, aceitei o convite, sem fazer ideia da incrível instituição que eu estava prestes a conhecer. 

## Quem tem fome tem pressa

A Ação da Cidadania foi fundada em 1993 pelo sociólogo Herbert de Souza, o Betinho. Formada por uma grande rede de comitês da sociedade civil organizada, contando com o apoio de lideranças locais e de diversos setores sociais, sua principal área de atuação é a luta contra a fome e a insegurança alimentar. Uma luta resumida precisamente em seu famoso lema: "quem tem fome tem pressa".

<div style="text-align: center;">
  <p><strong>Figura 1:</strong> Herbert de Souza, o Betinho</p>
  <img src="https://saude.abril.com.br/wp-content/uploads/2023/12/Saude-e-Pop_Betinho2_Acao-da-Cidadania_Divulgacao.jpg" width="400">
  <p style="font-size: 12px; color: #555;">
    Fonte: <a href="https://saude.abril.com.br/coluna/saude-e-pop/os-30-anos-do-natal-sem-fome-a-historia-de-herbert-de-souza-o-betinho/" target="_blank">
    saude.abril.com.br</a>. Acesso em: 7 abr. 2025
  </p>
</div>

## O Projeto

Minha missão era gerenciar o projeto "Gás para Todos", resultado de uma parceria entre a Ação da Cidadania e a Petrobras. Inicialmente, seriam 85.000 famílias atendidas em todo o território nacional, cada uma recebendo 5 vales-gás que poderiam ser trocados por botijões de GLP ao longo de um ano. Era, portanto, um projeto grandioso, que certamente iria impactar centenas de milhares de brasileiros em vulnerabilidade social. 

No entanto, precisaríamos cadastrar todas essas famílias; precisaríamos registrar todos os vales-gás recebidos por cada família; precisaríamos registrar os status de troca de todos os vales-gás. Ou seja, era uma quantidade gigantesca de dados para alguém que, como eu, tinha pouca experiência em colocar a matemática em prática fora das salas de aula.

E foi assim, aos tropeços, aprendendo na marra e na prática, que conheci a Ciência de Dados.

## A Cobertura

A Ação da Cidadania conta com a ajuda local de seus comitês para aumentar sua capilaridade. E foi com base na percepção da diretoria acerca da capacidade técnica desses comitês que as quantidades de famílias atendidas por UF foram arbitrariamente definidas. 

No entanto, naquele momento, tais decisões ainda careciam de dados objetivos. A instiuição ainda levava em consideração aspectos subjetivos demais em suas tomadas de decisão. A própria percepção da capacidade técnica dos comitês era baseada em impressões pessoais, em experiências passadas, e não em métricas, dados quantificáveis ou _benchmarking_.


## As Grandes Questões

Por tudo isso, ao longo de todo o projeto, eu sempre me perguntei:

**_"Estamos fazendo uma distribuição justa de recursos? As quantidades de famílias atendidas por UF refletem de forma precisa as diferenças de condições dessas regiões? Ou estamos inadvertidamente priorizando algumas delas, em detrimento de outras mais necessitadas? Nesse caso, quais seriam os dados objetivos que poderíamos usar para uma ditribuição ideal, buscando minimizar um dos maiores problemas do Brasil: a desigualdade social?"_**

São essas perguntas que pretendo responder ao longo do presente trabalho.

## O esquema de dados

Para podermos responder as perguntas acima, vamos precisar de uma tabela bastante simples, composta por somente três atributos. No entanto, temos alguns esforços necessários para consolidarmos o primeiro e o segundo deles, enquanto o terceiro será resultado de uma modelagem matemática proposta ao longo do trabalho.

Dessa forma, o seguinte esquema flat atenderia nossas necessidades:

![Esquema de dados](../Assets/scheme.png)

## No começo, o final

Vamos começar analisando os números finais do projeto.

Pouco mais de um ano depois de seu início, o projeto chegou ao fim. Foram inúmeros ajustes ao longo de seu ciclo de vida, algo normal em qualquer projeto de grande porte. 

Vários remanejamentos de vales-gás foram feitos entre os comitês, já que alguns mostravam-se menos capazes do que se pensava incialmente, enquanto outros estavam ávidos por receber mais recursos e direcioná-los a quem necessitava. Além disso, graças à readequação de despesas e ao hercúleo esforço de negociação de nosso setor financeiro com empresas fornecedoras de GLP, conseguimos aumentar a quantidade prevista inicialmente de 85.000 para 108.259 famílias atendidas. Além disso, para uma fração delas, conseguimos fazer duas entregas extras de vales-gás, totalizando 7 lotes, em vez dos 5 previstos inicialmente.

Vamos agora à carga dos dados.

## Preparando o Banco de Dados

Começamos dropando possíveis instâncias anteriores do Banco de Dados. Na sequência, levantamos o Banco de Dados para, assim, para trabalharmos com estruturas e dados consistentes.

In [0]:
%sql

DROP DATABASE IF EXISTS cobertura CASCADE;

CREATE DATABASE cobertura;

USE cobertura

## Fazendo as importações necessárias

Começamos importando o Pandas. 

Mas antes, uma observação: estou ciente da possibilidade de trabalharmos com _alias_. No entanto, ao longo de todo este trabalho, opto por não fazê-lo. Apesar de tornarem o código mais limpo, _alias_ também o tornam mais difícil de serem compreendidos, principalmente quando ainda estamos aprendendo a linguagem.


In [0]:
import pandas

## Carregando os dados dos beneficiários

Vamos hospedar na variável `csv_beneficiarios` o arquivo csv, hospedado no Google Drive, contendo informações acerca dos beneficiários. 

**Observação:** O arquivo csv aqui utilizado foi a base de nossa prestação de contas à Petrobras. Precisávamos fornecer dados acerca dos beneficiários atendidos, mas observando a LGPD. Daí termos dados pessoais dos beneficiários anonimizados. 

In [0]:

url = "https://drive.google.com/uc?export=download&id=1GSMAQ_DP-EVwO-Rx20QO1eJ8tE-Lpfry"
csv_beneficiarios = pandas.read_csv(url, delimiter=";",low_memory=False)

## Criando a tabela dos beneficiários

Uma vez carregados, vamos utilizar Spark para converter esses dados em um DataFrame, armazenado na variável `df_beneficiarios`. Em seguida, criamos a tabela  `beneficiarios_bronze`.

In [0]:
df_beneficiarios = spark.createDataFrame(csv_beneficiarios)
df_beneficiarios.write.mode("overwrite").saveAsTable("beneficiarios_bronze");

## Definindo a camada bronze dos dados

Chegamos, portanto, à camada bronze de nossos dados. A seguir, uma consulta SQL exibe seus primeiros registros.

In [0]:
%sql

SELECT 
  * 
FROM 
  beneficiarios_bronze
LIMIT 
  30

id,cpf,nome,endereco_completo,comite,lote
1e12fcca,008.***.***-**,D******** M*****,"*** **** ********* ********** **, Mãe Luiza, Natal, RN",Avoante,1o
4bb1a31c,006.***.***-**,M******* B****** D* A*****,"*** ***** ******* *********, **, *********** *******, Residencial Iolanda, Rio Branco, AC",Associação Olhar Diferente,1o
e1656944,359.***.***-**,F******** P****** M********,"*** ***** ******* *********, **, *********** *******, Residencial Iolanda, Rio Branco, AC",Associação Olhar Diferente,1o
3ae7e55d,042.***.***-**,K******** S**** D* S**** Q******,"*** ***** ******* *********, **, *********** *******, Residencial Iolanda, Rio Branco, AC",Associação Olhar Diferente,1o
2810dbbf,018.***.***-**,V****** F******* D* S****,"*** ***** ******* *********, **, *********** *******, Residencial Iolanda, Rio Branco, AC",Associação Olhar Diferente,1o
fe21dda7,465.***.***-**,M**** J***** P*******,"*** ***** *********, ***, **** ** ******, Seis De Agosto, Rio Branco, AC",Associação Olhar Diferente,1o
f2c18f76,058.***.***-**,M**** P******* C***** S*****,"*** ********, **, **** ** ******, Seis De Agosto, Rio Branco, AC",Associação Olhar Diferente,1o
ea7d3847,774.***.***-**,D******* R*****,"******** *********, **, **** ** ******, Seis De Agosto, Rio Branco, AC",Associação Olhar Diferente,1o
e775ba9b,579.***.***-**,F********* I***** D* S**** F****,"******** ********, ***, **** ** ******, Seis De Agosto, Rio Branco, AC",Associação Olhar Diferente,1o
3c8417ec,714.***.***-**,M**** D* L****** S**** D* S****,"******** ********, **, **** ** ******, Seis De Agosto, Rio Branco, AC",Associação Olhar Diferente,1o



## Definindo o catálogo de dados da camada bronze

Vamos conhecer melhor nossos dados através de seus metadados:

| Atributo | Tipo de Dado | Descrição | Domínio | Observações |
|---|---|---|---|---|
| id | Texto (UUID) | Identificador único do registro (Chave Primária) | 00000000 a ffffffff | Identificador hexadecimal de 8 dígitos gerado automaticamente |
| cpf | Texto | Cadastro de Pessoa Física do beneficiário |  | Dados parcialmente mascarados |
| nome | Texto | Nome do beneficiário |  | Dados parcialmente mascarados |
| endereco_completo | Texto | Endereço completo do beneficiário (logradouro, no., complemento, bairro, cidade, UF) | Restrito às cidades e Ufs definidas no escopo do projeto | Dados parcialmente mascarados |
| comite | Texto | Nome do comitê responsável pelo cadastro e acompanhamento do beneficiário | Restrito aos comitês participantes do projeto |  |
| lote | Texto | Lote no qual o beneficiário foi cadastrado no projeto | 1o, 2o, 3o, 4o, 5o, Extra 1, Extra 2 | Usado para logística/controle |





## Limpando os dados

Como podemos ver na camada bronze, há diversas informações irrelevantes para nossa análise, como `cpf`, `nome` e `lote`. Além disso, o atributo `endereco_completo` não está atomizado, contendo logradouro, número, complemento, bairro, município e UF do beneficiário. 

Para conseguirmos fazer nossa análise, precisamos obter o atributo `uf` a partir de `endereco_completo`. Além disso, prevendo a necessidade de futuras análises mais granuladas, podemos manter o atributo `comite`, além de obtermos o novo atributo `municipio`, também a partir de `endereco_completo`. 

Como todo endereço completo tem seus componentes separados por vírgula, além de apresentar município e UF como o penúltimo e o último elemento, respectivamente, podemos conjugar as funções `element_at()` e `split()` para obtermos os dois novos atributos `municipio` e `uf`.

## Definindo a camada prata dos dados

Chegamos, portanto, à camada prata de nossos dados. A seguir, uma consulta SQL exibe seus primeiros registros:

In [0]:
%sql

CREATE OR REPLACE TABLE beneficiarios_silver AS
SELECT
  id,
  comite,
  trim(element_at(split(endereco_completo, ','), -2)) AS municipio,
  trim(element_at(split(endereco_completo, ','), -1)) AS uf
FROM 
  beneficiarios_bronze
WHERE 
  endereco_completo IS NOT NULL;

SELECT 
  * 
FROM 
  beneficiarios_silver
ORDER BY 
  id
LIMIT
  30

id,comite,municipio,uf
000000ec,Conselho Quilombola,Vitoria da Conquista,BA
0000928f,Cáritas Piauí,Nossa Senhora dos Remédios,PI
0001f9f2,Comitê Curitiba,Piraquara,PR
0003bd67,Comitê Amazonas,Novo Airão,AM
0003db02,Comitê Amazonas,Novo Airão,AM
0004b92f,IMADEL,Guapimirim,RJ
0004f6c9,Comitê Maranhão,Tutóia,MA
0005d613,Arte e Vida,Arapongas,PR
00069ff8,INEC,Pacatuba,CE
00076e5f,Arebeldia,Belo Horizonte,MG




## Definindo o catálogo de dados da camada prata

Vamos conhecer melhor nossos dados através de seus metadados:

| Atributo | Tipo de Dado | Descrição | Domínio | Observações |
|---|---|---|---|---|
| id | Texto (UUID) | Identificador único do registro (Chave Primária) | 00000000 a ffffffff | Identificador hexadecimal de 8 dígitos gerado automaticamente |
| comite | Texto | Nome do comitê responsável pelo cadastro e acompanhamento do beneficiário | Restrito aos comitês participantes do projeto |  |
| municipio | Texto | Município obtido a partir do endereço completo | Restrito aos municípios definidos no escopo do projeto |  |
| uf | Texto | UF obtida a partir do endereço completo | AC, AL, AM, AP, BA, CE, DF, ES, GO, MA, MG, MS, MT, PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO |  |


## Criando uma nova view dos beneficiários agrupados por UF

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW beneficiarios_por_uf AS
SELECT
  uf,
  COUNT(*) AS publico_atendido
FROM beneficiarios_silver
GROUP BY uf;

SELECT * FROM beneficiarios_por_uf
ORDER BY uf

uf,publico_atendido
AC,824
AL,3835
AM,3833
AP,690
BA,17573
CE,6985
DF,1433
ES,791
GO,1623
MA,8981


Databricks visualization. Run in Databricks to view.

## Fazendo nossa primeira análise

Neste momento, já podemos fazer nossa primeira análise dos dados, verificando a distribuição de recursos do projeto por todo o território nacional.

![Distribuição real de recursos do projeto](../Assets/grafico_1.png)

## Real X Ideal

No gráfico acima, temos a distribuição real dos recrusos do projeto por UF. Mas, com base nas perguntas propostas, qual seria a distribuição ideal destes recursos?

Para isso, pensei em criar uma metodologia simples, utilizando dois indicadores: o IDH e a população total da UF. 

## Os Indicadores

O Índice de Desenvolvimento Humano (IDH) das UFs do Brasil é calculado pelo Programa das Nações Unidas para o Desenvolvimento (PNUD), em parceria com o Instituto de Pesquisa Econômica APlicada (IPEA) e a Fundação João Pinheiro. Seu último cálculo foi realizado em 2021.

As populações totais das UFs, por sua vez, podem ser obitdas a partir do último censo concluído, cujo ano de referência é 2022, realizado pelo Instituto Brasileiro de Geografia e Estatística (IBGE).

Mas como relacionar o público-alvo ideal, IDH e população total?

## Construindo a Modelagem Matemática


Vamos criar um modelo matemático para calcular o público-alvo ideal de cada UF. Para isso, vamos trabalhar com três variáveis:

### Variáveis:

$$
\text{ }\\
\left\{
\begin{array}{ll}
p \to \text{Público-alvo ideal que deveria ser atendido por UF (variável dependente);} \\
I \to \text{IDH da UF (variável independente);} \\
T \to \text{População total da UF (variável independente).}
\end{array}
\right.\\
\text{ }\\
$$

### Relações entre as variáveis:

Temos que $p$ e $I$ devem ser inversamente proporcionais, pois Ufs de **menor** IDH idealmente deveriam receber **maior** aporte de recursos. Já $p$ e $T$ devem ser diretamente proporcionais, pois UFs de **maior** população total deveriam receber **maior** aporte de recursos.

Resumindo:
$$
\text{ }\\
\left\{
    \begin{array}{ll}
    p \text{ e } I \to \text{Inversamente proporcionais;} \\
    p \text{ e } T \to \text{Diretamente proporcionais.}
\end{array}
\right.\\
\text{ }\\
$$

Assim, chegamos à relação abaixo, onde $K$ é uma constante de proporcionalidade:
$$
\text{ }\\
\frac{p \cdot I}{T}=K\\
\text{ }\\
$$

Essa relação pode ser reescrita da seguinte forma:
$$
\text{ }\\
\frac{p}{\frac{T}{I}}=K\\
\text{ }\\
$$

Ao termo $\frac{T}{I}$ daremos o nome de **Índice Beija-Flor**, representado por $B$, em homenagem à famosa fábula do Beija-Flor, que ilustra a importância de pequenos gestos no combate a grandes problemas.

Para saber mais: [A Fábula do Beija-Flor](https://www.youtube.com/watch?v=qow1qZNi1Og)

Temos, portanto:
$$
\begin{array}{cc}
\text{ }\\
\dfrac{p}{B}=K \\ 
\\ \text{ } \\
\fbox{$p=k \cdot B$} \\ 
\tiny {\text{(Fórmula 1)}}\\
\text{ }\\
\end{array}
$$

### Divisão proporcional:

Estamos prontos para fazer a divisão proporcional das famílias que idealmente deveriam ser atendidas por UF. Para isso, vamos supor que o somatório desses públicos-alvo seja igual ao total de famílias realmente atendidas pelo projeto, ou seja, 108.259:

$$
\text{ }\\
\left\{
    \begin{array}{ll}
    p_{\tiny{\text{AC}}}+p_{\tiny{\text{AL}}}+p_{\tiny{\text{AM}}}+p_{\tiny{\text{AP}}}+p_{\tiny{\text{BA}}}+p_{\tiny{\text{CE}}}+p_{\tiny{\text{DF}}}+\text{...}+p_{\tiny{\text{TO}}}=108.259 \tiny{\text{ (equação 1)}}\\
    \\ \text{ } \\
    \dfrac{p_{\tiny{\text{AC}}}}{B_{\tiny{\text{AC}}}}=\dfrac{p_{\tiny{\text{AL}}}}{B_{\tiny{\text{AL}}}}=\dfrac{p_{\tiny{\text{AM}}}}{B_{\tiny{\text{AM}}}}=\dfrac{p_{\tiny{\text{AP}}}}{B_{\tiny{\text{AP}}}}=\dfrac{p_{\tiny{\text{BA}}}}{B_{\tiny{\text{BA}}}}=\dfrac{p_{\tiny{\text{CE}}}}{B_{\tiny{\text{CE}}}}=\dfrac{p_{\tiny{\text{DF}}}}{B_{\tiny{\text{DF}}}}=...=\dfrac{p_{\tiny{\text{TO}}}}{B_{\tiny{\text{TO}}}}
    \end{array}
\right.\\
\text{ }\\
$$

Ora, sabemos que, em toda proporção, cada antecedente está para seu consequente assim como a soma dos antecedentes está para a soma dos consequentes. Ou seja:
$$
\text{ }\\
\dfrac{p_{\tiny{\text{AC}}}}{B_{\tiny{\text{AC}}}}=\dfrac{p_{\tiny{\text{AL}}}}{B_{\tiny{\text{AL}}}}=\dfrac{p_{\tiny{\text{AM}}}}{B_{\tiny{\text{AM}}}}=\dfrac{p_{\tiny{\text{AP}}}}{B_{\tiny{\text{AP}}}}=\dfrac{p_{\tiny{\text{BA}}}}{B_{\tiny{\text{BA}}}}=\dfrac{p_{\tiny{\text{CE}}}}{B_{\tiny{\text{CE}}}}=\dfrac{p_{\tiny{\text{DF}}}}{B_{\tiny{\text{DF}}}}=...=\dfrac{p_{\tiny{\text{TO}}}}{B_{\tiny{\text{TO}}}}=\frac{\Sigma p}{\Sigma B}\\
\text{ }\\
$$

Assim, deduzimos o valor da constante de proporcionalidade $K$:
$$
\text{ }\\
k=\frac{\Sigma p}{\Sigma B}\\
\text{ }\\
$$

Sabemos, da equação 1, que $\Sigma p = 108.259$. Mais à frente, depois que importarmos os dados do IBGE e os consolidarmos aos dados dos beneficiários, estaremos aptos a calcular o índice Beija-Flor ($B$) de cada UF e, consequentemente, seu somatório $\Sigma B$.

Dessa forma, podemos reescrever a fórmula 1, que nos permitirá calcular o público-alvo ideal $p$ de cada UF:
$$
\text{ }\\
\fbox{$p=\dfrac{108.259 \ \cdot \ B}{\Sigma B}$}\\
\text{ }\\
$$

O próximo passo, portanto, é carregar os dados do IBGE, obtendo IDH e população total de cada UF.

## Carregando os dados do IBGE

Vamos agora carregar um arquivo csv com as informações de IDH e população total, obtidas previamente no site do IBGE:

In [0]:
url = "https://drive.google.com/uc?export=download&id=1C0cSMvgx1GyCs2Jf-QhuXYJXR68wjbF2"
csv_ibge = pandas.read_csv(url,delimiter=";")

## Criando a tabela "ibge" 

Uma vez carregados, vamos utilizar Spark para converter esses dados em um DataFrame, armazenado na variável `df_ibge`. Em seguida, criamos a tabela `ibge`.

In [0]:
df_ibge = spark.createDataFrame(csv_ibge)
df_ibge.write.mode("overwrite").saveAsTable("ibge")

## Consolidando os beneficiários agrupados por UF com os dados do IBGE

Vamos agora consolidar os dados dos beneficiários com os dados obtidos do IBGE, criando também o atributo `indice_beija_flor`.

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW informacoes_consolidadas AS
SELECT
  beneficiarios_por_uf.uf,
  beneficiarios_por_uf.publico_atendido,
  ibge.populacao,
  ibge.idh,
  (cast(ibge.populacao AS DOUBLE) / cast(ibge.idh AS DOUBLE)) AS indice_beija_flor
FROM
  beneficiarios_por_uf
JOIN
  ibge
ON
  beneficiarios_por_uf.uf = ibge.uf;

SELECT 
  * 
FROM 
  informacoes_consolidadas
ORDER BY uf 

uf,publico_atendido,populacao,idh,indice_beija_flor
AC,824,830018,0.71,1169039.4366197183
AL,3835,3127683,0.684,4572635.96491228
AM,3833,3941613,0.7,5630875.714285715
AP,690,733759,0.688,1066510.1744186047
BA,17573,14141626,0.691,2.0465450072358903E7
CE,6985,8794957,0.734,1.1982230245231608E7
DF,1433,2817381,0.814,3461156.0196560198
ES,791,3833712,0.771,4972389.1050583655
GO,1623,7056495,0.737,9574620.081411127
MA,8981,6776699,0.676,1.0024702662721893E7


## Definindo a camada ouro dos dados

Chegamos, portanto, à camada ouro de nossos dados. A seguir, uma consulta SQL exibe seus registros:

In [0]:
%sql

CREATE OR REPLACE TABLE beneficiarios_gold AS 
WITH 
base AS (
  SELECT 
    SUM(indice_beija_flor) AS fator 
  FROM 
    informacoes_consolidadas
),
calculo AS (
  SELECT
    uf,
    publico_atendido,
    ROUND((108259 * indice_beija_flor / base.fator)) AS publico_alvo
  FROM informacoes_consolidadas, base
)
SELECT * FROM calculo;

SELECT
  *
FROM
  beneficiarios_gold
ORDER BY
  uf


uf,publico_atendido,publico_alvo
AC,824,468.0
AL,3835,1829.0
AM,3833,2252.0
AP,690,427.0
BA,17573,8186.0
CE,6985,4793.0
DF,1433,1384.0
ES,791,1989.0
GO,1623,3830.0
MA,8981,4010.0


## Definindo o catálogo de dados da camada ouro

Vamos conhecer melhor nossos dados através de seus metadados:

| Atributo | Tipo de Dado | Descrição | Domínio | Observações |
|---|---|---|---|---|
| uf | Texto | UF obtida a partir do endereço completo | AC, AL, AM, AP, BA, CE, DF, ES, GO, MA, MG, MS, MT, PA, PB, PE, PI, PR, RJ, RN, RO, RR, RS, SC, SE, SP, TO |  |
| publico_atendido | Inteiro | Quantidade de famílias que foram de fato atendidas pelo projeto | | Refere-se ao aspecto "real" do projeto |
| publico_alvo | Float | Quantidade de famílias que deveriam ter sido atendidas pelo projeto, com base na metodologia proposta | | Refere-se ao aspecto "ideal" do projeto |


## Fazendo nossa análise final

Finalmente podemos fazer nossa análise final dos dados, confrontando a distribuição real (azul) com a ideal (vermelho) de recursos do projeto por todo o território nacional.

![Distribuição real x ideal de recursos do projeto](../Assets/grafico_2.png)

## O que podemos concluir?

Somente DF e RR ficaram próximos de suas distribuições ideais. 

Podemos notar UFs que foram privilegiadas, como AC, AL, AM, AP, BA, CE, MA, PA, PB, PE, PI, PR, RN e SE. No caso específico de BA, a discrepância é gigantesca, tendo recebido muito mais recursos do que o ideal.

Mas então, podemos nos perguntar: existe excesso de ajuda?

Se os recursos são limitados, sim. Afinal, excesso de ajuda para alguns implica falta de ajuda para outros. Por isso, ES, GO, MG, MS, MT, RJ, RO, RS, SC, SP e TO receberam recursos aquém do ideal. No caso específico de SP, a discrepância é gigantesca, tendo recebido muito menos recursos do que o ideal. 

No caso do Rio de Janeiro, sede nacional da ONG, algo curioso: os recursos recebidos também ficaram bem aquém dos ideais. Os dados em si não nos mostram o motivo disto. Podemos apenas imaginar que tenha se tratado de um impulso altruísta em distribuir recursos para outras regiões do território nacional. Ou, talvez, questões ligadas à segurança pública, como a comercialização de botijões de gás controlada por narco-milicianos, tenham limitado a distribuição de recursos no estado. Um problema que foi de fato abordado em diversas reuniões de planejamento.

Enfim, quaisquer que tenham sido os motivos, podemos ver uma significativa diferença entre o aspecto real e o aspecto ideal do projeto.

## Mas isso significa que o projeto foi mal conduzido?

Certamente não!

No início, a referência para a distribuição de recursos foi a impressão subjetiva da capacidade técnica dos comitês. Podemos perceber que há comitês muito bem aparelhados para atender a população em vulnerabilidade social, enquanto outros carecem de uma estrutura que os permitam ser mais atuantes em suas comunidades. 

O comitê central da Ação da Cidadania poderia usar esses dados para propor estratégias de fortalecimento de alguns comitês. São Paulo, por exemplo, é um território gigantesco, com uma enorme população carente. Se a estrutura de seu comitê não é suficiente para o gerenciamento de mais recursos, algo deve ser feito no sentido de fomentar essa estrutura.

É claro que a metodologia aqui proposta pode ser discutida, ampliada ou reduzida. Mas o que realmente importa é que, utilizando dados objetivos, estratégias de médio e longo prazo podem ser definidas. E assim, a Ciência de Dados por ajudar a missão de nosso saudoso Betinho.

Dessa forma, quem sabe, um dia, em uma sociedade evoluída e bem estruturada, instituições como a Ação da Cidadania deixem de ser necessárias.

Mas até lá, quem tem fome tem pressa.